### Hypothesis 2: Firms with higher systemic risk experience higher default probabilities during market downturns.

# DELETE LATER

Basic idea:
- estimate beta of each firm vs market (probably use SPY returns for the market)
- define two market downturn periods as:
    -  Global Financial Crisis: September 1, 2008 to March 30, 2009
    -  COVID: February 20, 2020 to April 30, 2020
- calculate the mean pd during each crisis period
- look at correlation between beta and pd \
Expected result: positive and statistically significant correlation
